In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("../input/insurance/insurance.csv")

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
plt.figure(figsize=(12,6))
sns.displot(data=df,x='charges',kde=True)

In [ ]:
df['region'].unique()

In [ ]:
df.isna().sum()

In [ ]:
len(df)

# LET'S TRY LABEL ENCODING FIRST THEN WE CAN DO ONE-HOT-ENCODING

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
df['sex'] = le.fit_transform(df['sex'])

In [ ]:
df.head()

# female = 0
# male = 1

In [ ]:
df['smoker'] = le.fit_transform(df['smoker'])

In [ ]:
df.head()

# smoker:
# Yes = 1
# No = 0

In [ ]:
df['region'] = le.fit_transform(df['region'])

In [ ]:
df.head()

# Region: Southwest=3   ; Northwest=1   ; Southeast=2   ; Northeast=0

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.corr()['charges'].sort_values()[:-1]

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(data=df.corr(),annot=True,cmap='plasma')

In [ ]:
X = df.drop('charges',axis=1)
y = df['charges']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc = StandardScaler()

In [ ]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
X_train

# 1) LINEAR REGRESSION

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression(fit_intercept=True)

In [ ]:
lr.fit(X_train,y_train)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
lr_pred = lr.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error, r2_score

In [ ]:
(mean_squared_error(y_test,lr_pred))**0.5

# Bad Results with LR

# 2) CatBoost Regression

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
cb = CatBoostRegressor(random_state=42)

In [ ]:
cb.fit(X_train,y_train,use_best_model=True,eval_set=(X_test,y_test),early_stopping_rounds=30)

In [ ]:
cb_pred = cb.predict(X_test)

In [ ]:
(mean_squared_error(y_test,cb_pred))**0.5

In [ ]:
r2_score(y_test,cb_pred)

# Relatively better than LR

# 3) KNN Regression

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
knn = KNeighborsRegressor(n_neighbors=5)

In [ ]:
knn.fit(X_train,y_train)

In [ ]:
knn_pred = knn.predict(X_test)

In [ ]:
(mean_squared_error(y_test,knn_pred))**0.5

# KNN better than LR but worse than CB

# 4) SVR

In [ ]:
from sklearn.svm import SVR

In [ ]:
svr = SVR()

In [ ]:
svr.fit(X_train,y_train)

In [ ]:
svr_pred = svr.predict(X_test)

In [ ]:
(mean_squared_error(y_test,svr_pred))**0.5

# SVR worst

# 5) Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(random_state=42,n_estimators=100)

In [ ]:
rf.fit(X_train,y_train)

In [ ]:
rf_pred = rf.predict(X_test)

In [ ]:
(mean_squared_error(y_test,rf_pred))**0.5

# RF 2nd best after CB

# So the best method was CatBoost Regression

# NOW LET'S TRY IMPLEMENTING ONE-HOT ENCODING

In [ ]:
df2 = pd.read_csv("../input/insurance/insurance.csv")

In [ ]:
df2.head()

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ct = ColumnTransformer(transformers = [('encoder',OneHotEncoder() , [1,3,4,5])],remainder='passthrough')

In [ ]:
X2 = df.drop('charges',axis=1)
y2 = df['charges']
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X2, y2, test_size=0.3, random_state=42)

In [ ]:
X_train_2 = np.array(ct.fit_transform(X_train_2))
X_test_2 = np.array(ct.transform(X_test_2))

In [ ]:
X_train_2 = sc.fit_transform(X_train_2)
X_test_2 = sc.transform(X_test_2)

In [ ]:
X_train_2

In [ ]:
X_train_2.shape

In [ ]:
X_test_2.shape

# 1) Linear Regression

In [ ]:
lr2 = LinearRegression()

In [ ]:
lr2.fit(X_train_2,y_train_2)

In [ ]:
lr2_pred = lr2.predict(X_test_2)

In [ ]:
(mean_squared_error(y_test_2,lr2_pred))**0.5

# 2) CatBoost Regression

In [ ]:
cb2 = CatBoostRegressor(random_state=42)

In [ ]:
cb2.fit(X_train_2,y_train_2,use_best_model=True,eval_set=(X_test_2,y_test_2),early_stopping_rounds=30,plot=True)

In [ ]:
cb2_pred = cb2.predict(X_test_2)

In [ ]:
(mean_squared_error(y_test_2,cb2_pred))**0.5

# ONE HOT ENCODING MADE THE MODEL A BIT MORE WORSE

In [ ]:
df.describe()

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(y_test_2,cb2_pred)

# BEST PRACTICE FOR THIS CASE WOULD BE TO USE CB WITH LABEL ENCODING